In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
data = pd.read_csv('dataset/kc_house_data.csv')
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180.0,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170.0,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770.0,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050.0,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680.0,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
def remove_outliers(data, column):
        
    # Defining the quartiles
    Q1 = np.quantile(data[column], 0.25, interpolation = 'midpoint')
    # Q2 = np.quantile(data[column], 0.50, interpolation = 'midpoint')
    Q3 = np.quantile(data[column], 0.75, interpolation = 'midpoint') 
    FIQ = Q3 - Q1
    
    # Removing lowers outliers
    data = data.loc[data[column] > ( Q1 - 1.5*FIQ )]

    # Removing highers outliers
    data = data.loc[data[column] < ( Q3 + 1.5*FIQ )]
    
    data.reset_index(inplace = True)
    data.drop(columns = 'index', axis = 1, inplace= True)
    return data

data = remove_outliers(data, 'price')

def take_houses(data, house_data,H):
    sample = pd.DataFrame()
    data_aux = pd.DataFrame()
    for dado in house_data:
        data_aux = data.loc[data['id'] == dado[3]]
        sample = pd.concat([sample, data_aux])
        
    print('Este é o tamnho da amostra = ',len(sample))
    sample.drop_duplicates(inplace = True)
    print('Este é o tamnho da amostra = ',len(sample))
    sample.reset_index(inplace = True)
    for i in range(len(sample)):
        for dado in house_data:
            if sample.loc[i, 'id'] == dado[3]:
                sample.loc[i,'median_price'] = dado[2]
                sample.loc[i,'estimate_profit'] = dado[0]
                sample.loc[i,'estimate_profit%'] = '%.1f' %dado[1]
                sample.loc[i,'x%_lower'] = '%.1f' %(100*(1 - (sample.loc[i,'price']/dado[2])))
            
            
    sample.dropna(axis = 0, inplace = True)
    sample.reset_index(inplace = True)
    sample['hypothesis'] = H
    sample.drop(['index'], axis = 1, inplace = True)
    return sample

def show_profit_estimation(data):
    data['estimate_profit%'] = data['estimate_profit%'].astype(float)
    print('Profit estimation')
    print('Minimal estimated profit, %.1f' %data['estimate_profit%'].min(),'%')
    print('Maximum estimated profit, %.1f' %data['estimate_profit%'].max(),'%')
    print('Maximum estimated profit, %.1f' %data['estimate_profit%'].mean(),'%')

In [5]:
H1 = 'Houses with waterfront are, in average, 20% more expensive.'
print('H1 - ' + H1)

# Select data where the houses have waterfront
# Take the averaged price per zipcode and condition
data_wf = data.loc[data['waterfront'] == 1, ['condition', 'zipcode', 'price']].groupby(['condition', 'zipcode']).median().reset_index()
data_wf.columns = ['condition', 'zipcode', 'median_price']

# Select data where the houses haven't waterfront
# Take the averaged price per zipcode and condition
data_nwf = data.loc[data['waterfront'] == 0, ['condition', 'zipcode', 'price']].groupby(['condition', 'zipcode']).median().reset_index()
data_nwf.columns = ['condition', 'zipcode', 'median_price']

# Compare the price for houses with the same condition and region
diff_price = np.array([])
for i in range(len(data_wf)):
    for k in range(len(data_nwf)):
        if (data_wf.loc[i,'zipcode'] == data_nwf.loc[k,'zipcode']) & (data_wf.loc[i,'condition'] == data_nwf.loc[k,'condition']):
            diff_price = np.append(diff_price, 100*(data_wf.loc[i,'median_price']/data_nwf.loc[k, 'median_price'] - 1))

print(" - Value found: %.1f" %(sum(diff_price)/len(diff_price)),'%')
print(" - H1 confirmed")
print('Actilly the prices of houses with waterfront are, in the average, about 97% higher than prices of houses without waterfront.')
print('Lets check price distribution for houses with waterfront')

data_aux = data.loc[data['waterfront'] == 1]
fig = px.histogram(data_aux, x = 'price')
fig.update_layout(
    font_size = 20,
    title = 'Price distribution for houses with waterfront',
    xaxis_title = 'Price (USD)',
    yaxis_title = 'Number of houses'
)
fig.show()

print('The recommendation is to buy houses with prices lower %.1f, the the median price for houses with waterfront.' %data.loc[data['waterfront'] == 1 ,'price'].median())

data_aux.reset_index(inplace = True)
data_aux.drop(columns = 'index', inplace = True)   
house_data = []
for i in range(len(data_wf)):
    for k in range(len(data_aux)):
        if (data_wf.loc[i, 'median_price'] > data_aux.loc[k, 'price']) & (data_wf.loc[i,'zipcode'] == data_aux.loc[k,'zipcode']) & (data_wf.loc[i,'condition'] == data_aux.loc[k,'condition']):
            house_data.append([(data_wf.loc[i,'median_price'] - data_aux.loc[k, 'price']),100*(data_wf.loc[i,'median_price']/data_aux.loc[k, 'price'] - 1), data_wf.loc[i, 'median_price'],data_aux.loc[k, 'id']])

sample_h1 = take_houses(data, house_data,'H1')
print(len(sample_h1))
show_profit_estimation(sample_h1)
sample_h1.head()

H1 - Houses with waterfront are, in average, 20% more expensive.
 - Value found: 100.2 %
 - H1 confirmed
Actilly the prices of houses with waterfront are, in the average, about 97% higher than prices of houses without waterfront.
Lets check price distribution for houses with waterfront


The recommendation is to buy houses with prices lower 650000.0, the the median price for houses with waterfront.


/home/eliel/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Este é o tamnho da amostra =  24
Este é o tamnho da amostra =  24
24
Profit estimation
Minimal estimated profit, 2.5 %
Maximum estimated profit, 112.1 %
Maximum estimated profit, 25.3 %


,level_0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,zipcode,lat,long,sqft_living15,sqft_lot15,median_price,estimate_profit,estimate_profit%,x%_lower,hypothesis
0,0,121039042,20150313T000000,425000.0,3,2.75,3610,107386,1.5,1,...,98023,47.3351,-122.362,2630,42126,464000.0,39000.0,9.2,8.4,H1
1,1,2923039243,20141113T000000,340000.0,4,1.00,1200,11834,1.0,1,...,98070,47.4557,-122.443,1670,47462,370000.0,30000.0,8.8,8.1,H1
2,2,2781600195,20141117T000000,285000.0,1,1.00,1060,54846,1.0,1,...,98070,47.4716,-122.445,2258,31762,370000.0,85000.0,29.8,23.0,H1
3,3,7129303070,20140820T000000,735000.0,4,2.75,3040,2415,2.0,1,...,98118,47.5188,-122.256,2620,2433,842475.0,107475.0,14.6,12.8,H1
4,4,6329000050,20150310T000000,641500.0,1,1.00,1000,9084,1.0,1,...,98146,47.5007,-122.382,1090,6536,695750.0,54250.0,8.5,7.8,H1


In [6]:
import plotly.express as px
H5 = 'House that have more than one bathroom are, in average, 15% more expensive.'
print('H5 - ' + H5)

# Select houses with one bathroom
# Take the averaged price per zipcode and condition
data_onebathroom = data.loc[data['bathrooms'] <= 1, ['condition', 'zipcode', 'price']].groupby(['condition', 'zipcode']).median().reset_index()
data_onebathroom.columns = ['condition', 'zipcode', 'median_price']

# Select houses with more than one bathroo
# Take the averaged price per zipcode and condition
data_m_onebathrooms = data.loc[data['bathrooms'] > 1, ['condition', 'zipcode', 'price']].groupby(['condition', 'zipcode']).median().reset_index()
data_m_onebathrooms.columns = ['condition', 'zipcode', 'median_price']

profit = []
for i in range(len(data_onebathroom)):
    for k in range(len(data_m_onebathrooms)):
        if (data_onebathroom.loc[i,'zipcode'] == data_m_onebathrooms.loc[k,'zipcode']) & (data_onebathroom.loc[i,'condition'] == data_m_onebathrooms.loc[k,'condition']):
            profit.append(100*(data_m_onebathrooms.loc[k, 'median_price']/data_onebathroom.loc[i, 'median_price'] - 1))

avg_price_ratio = pd.DataFrame()
avg_price_ratio['avg_price_ratio'] = profit
avg_price_ratio.dropna(axis = 0, inplace= True)
print(' - Value found: %.1f' % avg_price_ratio['avg_price_ratio'].mean(),'%')
print(" - H5 confirmed")

print('Houses with more than one bathrooms are, in average, about 44% more expensive')
print('Lets see how many houses there are in the portifolio for each amount of bathrooms.')

bathrooms = data['bathrooms'].unique().tolist()
bath_count = data['bathrooms'].value_counts().to_frame().reset_index()
bath_count.columns = ['bathrooms','count']

fig = px.bar(bath_count, x = 'bathrooms', y = 'count')
fig.update_layout(
    font_size = 20,
    title = 'Amount of houses for each number of bathrooms',
    xaxis_title = 'Number of bathrooms',
    yaxis_title = 'Number of houses'
)
fig.show()
print("From the chart, we can see that the amount of houses where the number of bathrooms is higher or equal 1 and less or equal than 2.25 represents the major part of the houses. That indicate people prefere these houses. So the recommendation is to buy this kind of houses, due to will easier to trade. Let's estimate the profit by trading with these kind of houses.")

data_m_onebathrooms = data.loc[(data['bathrooms'] >= 1) & (data['bathrooms'] <= 2.5), ['bathrooms','condition','zipcode','price']].groupby(['bathrooms', 'condition', 'zipcode']).median().reset_index()
data_m_onebathrooms.columns = ['bathrooms','condition', 'zipcode', 'median_price']
bathrooms = data_m_onebathrooms['bathrooms'].unique().tolist()

print(data.loc[data['bathrooms'] == bathrooms[0],'id'].count())
print(data_m_onebathrooms.loc[data_m_onebathrooms['bathrooms'] == bathrooms[0],'bathrooms'].count())
bath_count.head()

house_data.clear()
for bathroom in bathrooms:
    data_aux_1 = data.loc[data['bathrooms'] == bathroom].reset_index() 
    data_aux_2 = data_m_onebathrooms.loc[data_m_onebathrooms['bathrooms'] == bathroom].reset_index()
    for i in range(len(data_aux_2)):
        for k in range(len(data_aux_1)):
            if (data_aux_2.loc[i, 'median_price'] > data_aux_1.loc[k, 'price']) & (data_aux_2.loc[i,'zipcode'] == data_aux_1.loc[k,'zipcode']) & (data_aux_2.loc[i,'condition'] == data_aux_1.loc[k,'condition']):
                house_data.append([(data_aux_2.loc[i,'median_price'] - data_aux_1.loc[k, 'price']),100*(data_aux_2.loc[i, 'median_price']/data_aux_1.loc[k, 'price'] - 1), data_aux_2.loc[i, 'median_price'],data_aux_1.loc[k, 'id']])


H5 - House that have more than one bathroom are, in average, 15% more expensive.
 - Value found: 42.2 %
 - H5 confirmed
Houses with more than one bathrooms are, in average, about 44% more expensive
Lets see how many houses there are in the portifolio for each amount of bathrooms.


From the chart, we can see that the amount of houses where the number of bathrooms is higher or equal 1 and less or equal than 2.25 represents the major part of the houses. That indicate people prefere these houses. So the recommendation is to buy this kind of houses, due to will easier to trade. Let's estimate the profit by trading with these kind of houses.
3847
245


In [7]:
print(len(house_data))
sample_h5 = take_houses(data, house_data,'H5')
show_profit_estimation(sample_h5)
sample_h5.head()

8220
Este é o tamnho da amostra =  8399
Este é o tamnho da amostra =  8315
Profit estimation
Minimal estimated profit, 0.0 %
Maximum estimated profit, 299.6 %
Maximum estimated profit, 20.6 %


,level_0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,zipcode,lat,long,sqft_living15,sqft_lot15,median_price,estimate_profit,estimate_profit%,x%_lower,hypothesis
0,0,9272202260,20140924T000000,130000.0,3,1.0,1200,7000,2.0,0,...,98116,47.5883,-122.384,3290,6000,210000.0,80000.0,61.5,38.1,H5
1,1,2023049218,20140716T000000,105500.0,2,1.0,930,7740,1.0,0,...,98148,47.4611,-122.324,1620,8584,275250.0,169750.0,160.9,61.7,H5
2,2,2023049218,20150316T000000,445000.0,2,1.0,930,7740,1.0,0,...,98148,47.4611,-122.324,1620,8584,275250.0,169750.0,160.9,-61.7,H5
3,3,40000362,20140506T000000,78000.0,2,1.0,780,16344,1.0,0,...,98168,47.4739,-122.280,1700,10387,79500.0,1500.0,1.9,1.9,H5
4,4,7895500070,20150213T000000,240000.0,4,1.0,1220,8075,1.0,0,...,98001,47.3341,-122.282,1290,7800,254975.0,14975.0,6.2,5.9,H5


In [8]:
#H8
data_below_trend_line = data.loc[data['price'] < (data['sqft_living']*167.3602 + 146400), ['id', 'sqft_living','condition', 'zipcode', 'price']].reset_index()
house_data = []
conditions = data['condition'].unique().tolist()
zipcodes = data['zipcode'].unique().tolist()
for condition in conditions:
    for zipcode in zipcodes:
        data_aux = data_below_trend_line.loc[(data_below_trend_line['condition'] == condition) & (data_below_trend_line['zipcode'] == zipcode)].reset_index()
        for i in range(len(data_aux)):
            house_data.append([data_aux.loc[i,'sqft_living']*167.3602 + 146400 - data_aux.loc[i, 'price'], 100*((data_aux.loc[i,'sqft_living']*167.3602 + 146400)/data_aux.loc[i, 'price'] - 1),data_aux.loc[i,'sqft_living']*167.3602 + 146400, data_aux.loc[i, 'id']])   



In [9]:
sample_h8 = take_houses(data, house_data, 'H8')
sample_h8.head()

Este é o tamnho da amostra =  11104
Este é o tamnho da amostra =  10928


,level_0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,zipcode,lat,long,sqft_living15,sqft_lot15,median_price,estimate_profit,estimate_profit%,x%_lower,hypothesis
0,0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,...,98178,47.5112,-122.257,1340,5650,343885.036,121985.036,55.0,35.5,H8
1,1,4058801670,20140717T000000,445000.0,3,2.25,2100,8201,1.0,0,...,98178,47.5091,-122.244,2660,8712,497856.420,52856.420,11.9,10.6,H8
2,2,2976800796,20140925T000000,236000.0,3,1.00,1300,5898,1.0,0,...,98178,47.5053,-122.255,1320,7619,363968.260,127968.260,54.2,35.2,H8
3,3,6874200960,20150227T000000,170000.0,2,1.00,860,5265,1.0,0,...,98178,47.5048,-122.272,1650,8775,290329.772,120329.772,70.8,41.4,H8
4,4,4268200055,20150501T000000,245000.0,3,1.75,1740,11547,1.0,0,...,98178,47.4945,-122.220,880,78408,437606.748,192606.748,78.6,44.0,H8


In [11]:
show_profit_estimation(sample_h8)

Profit estimation
Minimal estimated profit, 0.0 %
Maximum estimated profit, 257.9 %
Maximum estimated profit, 42.5 %


In [28]:
#H9
data_zip = data[['price', 'condition', 'zipcode']].groupby(['condition', 'zipcode']).median().reset_index()
data_zip.columns = ['condition', 'zipcode', 'median_price']
print(data_zip.shape)
houses_2010 =  data.loc[(data['yr_built'] >= 2010), ['id', 'price', 'condition', 'zipcode']].reset_index()
print(houses_2010.shape)

house_data = []
for i in range(len(houses_2010)):
    for k in range(len(data_zip)):
        if (houses_2010.loc[i, 'condition'] == data_zip.loc[k, 'condition']) & (houses_2010.loc[i, 'zipcode'] == data_zip.loc[k, 'zipcode']) & (houses_2010.loc[i, 'price'] < data_zip.loc[k, 'median_price']):
            trade.append(100*(data_zip.loc[k, 'median_price']/houses_2010.loc[i, 'price'] - 1))
            house_data.append([data_zip.loc[k, 'median_price'] - houses_2010.loc[i, 'price'],100*(data_zip.loc[k, 'median_price']/houses_2010.loc[i, 'price'] - 1), data_zip.loc[k, 'median_price'], houses_2010.loc[i, 'id'] ])    
   

(280, 3)
(1124, 5)


In [29]:
# Sample of general data analysis
sample_h9 = take_houses(data, house_data, 'H()')
sample_h9.head()
show_profit_estimation(sample_h9)    


Este é o tamnho da amostra =  284
Este é o tamnho da amostra =  284
Profit estimation
Minimal estimated profit, 0.0 %
Maximum estimated profit, 169.7 %
Maximum estimated profit, 20.7 %


In [82]:
print(sample_h1.shape)
sample_h1.sort_values('estimate_profit%', inplace = True, ascending = False)
houses_to_buy = sample_h1[['id','price','median_price','estimate_profit','estimate_profit%','hypothesis']]
houses_to_buy.to_csv('dataset/sample_h1.csv', index = False)
houses_to_buy

(24, 26)


,id,price,median_price,estimate_profit,estimate_profit%,hypothesis
12,3523029041,290000.0,615000.0,325000.0,112.1,H1
11,2013802030,357000.0,650000.0,293000.0,82.1,H1
8,7631800110,380000.0,685000.0,305000.0,80.3,H1
17,5216200090,385000.0,615000.0,230000.0,59.7,H1
2,2781600195,285000.0,370000.0,85000.0,29.8,H1
5,6329000185,540000.0,695750.0,155750.0,28.8,H1
13,1922039062,480000.0,615000.0,135000.0,28.1,H1
23,222029026,340000.0,429592.0,89592.0,26.4,H1
19,2013802060,500000.0,597500.0,97500.0,19.5,H1
16,7003200120,528000.0,615000.0,87000.0,16.5,H1


In [79]:
print(sample_h5.shape)
sample_h5.sort_values('estimate_profit%', inplace = True, ascending = False)
houses_to_buy = sample_h5[['id','price','median_price','estimate_profit','estimate_profit%','hypothesis']].head(50)
houses_to_buy.to_csv('dataset/sample_h5.csv', index = False)
sample_h5.head()

(8314, 26)
(50, 6)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,sqft_living15,sqft_lot15,median_price,estimate_profit,estimate_profit%,x%_lower,hypothesis
0,1238500451,20150209T000000,130000.0,3,1.0,1110,7520,1.0,0,0,...,98033,47.6830,-122.176,1440,8400,519500.0,389500.0,299.6,75.0,H5
1,6840700165,20140701T000000,202000.0,1,1.0,590,833,1.0,0,0,...,98122,47.6082,-122.299,780,1617,800000.0,598000.0,296.0,74.8,H5
2,1049010620,20140513T000000,90000.0,2,1.0,790,2640,1.0,0,0,...,98034,47.7351,-122.178,1310,2064,335000.0,245000.0,272.2,73.1,H5
3,5016003230,20150218T000000,169317.0,2,1.0,790,4000,1.0,0,2,...,98112,47.6248,-122.301,1700,4200,600000.0,430683.0,254.4,71.8,H5
4,2870000040,20141110T000000,145000.0,2,1.0,800,8125,1.0,0,0,...,98033,47.6836,-122.174,2390,8125,460500.0,315500.0,217.6,68.5,H5


In [81]:
print(sample_h8.shape)
sample_h8.sort_values('estimate_profit%', inplace = True, ascending = False)
houses_to_buy = sample_h8[['id','price','median_price','estimate_profit','estimate_profit%','hypothesis']].head(50)
houses_to_buy.to_csv('dataset/sample_h8.csv', index = False)
sample_h8.head()

(10927, 26)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,sqft_living15,sqft_lot15,median_price,estimate_profit,estimate_profit%,x%_lower,hypothesis
5583,7999600180,20140529T000000,83000.0,2,1.0,900,8580,1.0,0,0,...,98168,47.4727,-122.270,2060,6533,297024.180,214024.180,257.9,72.1,H8
10789,40000362,20140506T000000,78000.0,2,1.0,780,16344,1.0,0,0,...,98168,47.4739,-122.280,1700,10387,276940.956,198940.956,255.1,71.8,H8
5088,1322049150,20150305T000000,85000.0,2,1.0,910,9753,1.0,0,0,...,98032,47.3897,-122.236,1160,7405,298697.782,213697.782,251.4,71.5,H8
8729,4239400300,20141129T000000,90000.0,3,1.0,980,2490,2.0,0,0,...,98092,47.3170,-122.182,980,3154,310412.996,220412.996,244.9,71.0,H8
5957,3421079032,20150217T000000,75000.0,1,0.0,670,43377,1.0,0,0,...,98022,47.2638,-121.906,1160,42882,258531.334,183531.334,244.7,71.0,H8


In [69]:
print(sample_h9.shape)
sample_h9.sort_values('estimate_profit%', inplace = True, ascending = False)
houses_to_buy = sample_h9[['id','price','median_price','estimate_profit','estimate_profit%','hypothesis']]
houses_to_buy.to_csv('dataset/sample_h9.csv', index = False)
sample_h9.head()

(284, 26)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,sqft_living15,sqft_lot15,median_price,estimate_profit,estimate_profit%,x%_lower,hypothesis
75,2626119062,20141112T000000,155000.0,3,1.00,1300,6098,1.0,0,0,...,98014,47.7074,-121.364,1300,6849,418000.0,263000.0,169.7,62.9,H()
231,7242800020,20140815T000000,277140.0,3,1.50,1190,785,2.0,0,0,...,98052,47.6781,-122.117,2820,5626,674000.0,396860.0,143.2,58.9,H()
248,5528600005,20150327T000000,272167.0,2,2.50,1620,3795,2.0,0,0,...,98027,47.5321,-122.034,1620,6000,580000.0,307833.0,113.1,53.1,H()
283,844000965,20140626T000000,224000.0,3,1.75,1500,11968,1.0,0,0,...,98010,47.3095,-122.002,1320,11303,460000.0,236000.0,105.4,51.3,H()
145,6056100150,20140623T000000,160797.0,3,1.50,1270,2356,2.0,0,0,...,98108,47.5671,-122.298,1490,2175,330000.0,169203.0,105.2,51.3,H()


In [52]:
houses_to_buy = sample_h1[['id','price','median_price', 'estimate_profit','estimate_profit%','hypothesis']]
houses_to_buy.head()


,id,price,median_price,estimate_profit,estimate_profit%,hypothesis
12,3523029041,290000.0,615000.0,325000.0,112.1,H1
8,7631800110,380000.0,685000.0,305000.0,80.3,H1
11,2013802030,357000.0,650000.0,293000.0,82.1,H1
17,5216200090,385000.0,615000.0,230000.0,59.7,H1
5,6329000185,540000.0,695750.0,155750.0,28.8,H1
